In [1]:
from traffic.core import Traffic, Flight
import pandas as pd
import time

In [2]:
flights = Traffic.from_file("Data/states_2022-06-27-17.csv")
flights.apply_time

<bound method lazy_evaluation.<locals>.wrapper.<locals>.λf of    icao24  callsign  count
0  502d28  BTI49E      360
1  4ac961  NVR743      360
2  a4e7fb  PXT415      360
3  4009c7  BAW861      360
4  000001  TGJSC       359
5  501c13  SDR59WT     359
6  500241  T7MCV       359
7  500233  T7CLB       359
8  500217  T7MASHI     359
9  500205  MEA203      359>

In [3]:
while flight := flights.next():
    flight: Flight = flight
    print(flight.data.shape)
    time.sleep(1)

.eval() has been automatically appended for you.
Check the documentation for more options.


UFuncTypeError: ufunc 'greater' did not contain a loop with signature matching types (<class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.TimeDelta64DType'>) -> None

In [ ]:
def fix_flight(f):
    f.data.loc[:, 'timestamp'] = pd.to_datetime(f.data.['timestamp'], unit='s').dt.tz_localize('UTC')

In [ ]:
flight.data["timestamp"]

In [ ]:
flight

In [2]:
from traffic.data.samples import quickstart
from traffic.data import airports

In [3]:
quickstart.airborne()

,,count
icao24,callsign,
39b002,FHMAC,3283
3aabfc,FMY8055,2133
39c82b,PEA501,1891
4bc844,PGT90Y,1599
4d02be,JFA12P,1591
3d7009,DFORH,1559
3985a4,AFR19BH,1547
3944f1,AFR15AH,1546
3999e4,PEA302,1541


In [4]:
quickstart.query('callsign.str.startswith("AFR")').first("10 min").eval()

,,count
icao24,callsign,
394c13,AFR1753,600
3951c1,AFR6724,600
398567,AFR15XV,600
398564,AFR9455,600
3965af,AFR276,600
3965a5,AFR4145,600
3965a3,AFR072,600
39644d,AFR914,600
396441,AFR104,600


In [5]:
def select_landing(airport: "Airport"):
    for flight in quickstart:
        if low_alt := flight.query("altitude < 3000"):         # Flight -> None or Flight
            if low_alt.last("2 min").vertical_rate_mean < -500:# Flight -> bool
                if low_alt.first("2 min").vertical_rate_mean < 500:# Flight -> bool
                    if low_alt.intersects(airport):                # Flight -> bool
                        if low_alt.aligned_on_ils(airport).has():  # Flight -> bool
                            yield low_alt          # Flight -> None or Flight

In [6]:
Traffic.from_flights(select_landing(airports["LFPO"]))[3].map_leaflet()

Map(center=[48.69000510556506, 2.232647785496943], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [7]:
quickstart["3964f4"].map_leaflet()

Several callsigns for one flight, consider splitting
Several callsigns for one flight, consider splitting


Map(center=[48.537532136850984, 2.335286307940885], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [8]:
from traffic.data.datasets import squawk7700

In [9]:
ids = squawk7700.query("diverted == diverted").flight_ids

In [10]:
from ipywidgets import Layout
squawk7700[ids[4]].map_leaflet(
    highlight=dict(red=lambda f: f.emergency()),
    layout=Layout(height="500px", max_width="800px"),
)

Map(center=[39.61573054457169, -75.24646557692384], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [11]:
squawk7700[ids[4]].take

AttributeError: 'Flight' has no attribute 'take'